In [ ]:
import duckdb
import pandas as pd
import json

# Support Function

In [ ]:
def extract_field_values(field_values, key):
    try:
        values = json.loads(field_values)
        for item in values:
            if isinstance(item, dict) and item.get("key") == key:
                return item.get("value")
    except json.JSONDecodeError:
        pass
    return None

In [ ]:
def extract_transaction_source(properties_file_path):
    print('Extracting transaction data source...', end='')
    conn = duckdb.connect(database=":memory:")
    try:
        query = f"SELECT * FROM '{properties_file_path}'"
        df = conn.execute(query).fetchdf()
        # df.to_csv('pure_datasources.csv', index=False)

        transaction_schema = list()
        with open('transaction_schema.csv') as file:
            rows = csv.reader(file)
            for row in rows:
                transaction_schema.append(row[0])

        transaction_df = pd.DataFrame()
        for field in transaction_schema:
            transaction_df[field] = df['field_values'].apply(lambda x: extract_field_values(x, field))

        transaction_df.to_csv('transaction_source.csv', index=False)

        print('Done')
        return transaction_df

    except Exception as e:
        print(f"Error processing data: {e}")
        return None

    finally:
        conn.close()

In [ ]:
def get_record(json_field):
    record = dict()
    record[json_field['label']] = json_field['value']

    return record 

# Exploratory Data Analytics

In [ ]:
conn = duckdb.connect(database="listing.db")

In [ ]:
query = f"SELECT * FROM 'all_properties.parquet'"

In [ ]:
df = conn.execute(query).fetchdf()

In [ ]:
df.info()

In [ ]:
df['json_field_values'] = df['field_values'].apply(json.loads)

In [ ]:
def get_field_record(json_field_value):
    record = dict()
    for i in range(len(json_field_value)):
        record[json_field_value[i]['label']] = json_field_value[i]['value']

    return record

In [ ]:
df['records'] = df['json_field_values'].apply(get_field_record)

In [ ]:
records = df['records'].to_list()

In [ ]:
df = df[['id', 'records']]

In [ ]:
df = pd.DataFrame.from_records(df['records'])

In [ ]:
df.to_csv('transaction_source.csv', index=False)